## Notebook for use case digital posting assistant - stage1
### Module A vectorize accounting assignment guide
#### Ojectives
- In this module we will develop the load and the vectorization of the text-file for the accounting assignment guide
- The vectorized accounting assignemnt guide will finaly stored in a SAP HANA vector database

#### Processing steps from concept
A0 - preparation

A2 - load and splitt: load the pdf-file containing the accounting assignment guide data from a folder and splitt the data into text_chunks

A3 - vectorize and embedd: vectorize the splitted data with embedding function. Use an embedding function to convert the text chunks into vector representations

A4 - store: create/clear a sap hana database - table and store the vector in this table

### A0 - Setup and configuration Modul A

The following setup-steps where processed:

* A0.0 Start SAP instances
* A0.1 install py-packages
* A0.2 load env-variables from config.json-file
* A0.3 Setup and test connection to HANA DB
* A0.4 Setup LLM-Connection to SAP AI-HUB


### A0.0 Start SAP Instances

* Start BTP Cockpit
* Start SAP Build Dev Space
* Start HANA DB

In [ ]:
# A0.1 install py-packages
# RESET KERNEL AFTER INSTALLATION

%pip install --upgrade pip

%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install folium --break-system-packages
%pip install ipywidgets --break-system-packages
%pip install pypdf
%pip install -U ipykernel
%pip install hana-ml
%pip install langchain
%pip install hdbcli
%pip install sqlalchemy-hana

In [ ]:
# A0.2 load env-variables from config.json-file
# This script loads environment variables from a JSON configuration file
# and sets them in the current environment. It raises an error if the file does not exist
# or if the JSON file is malformed.

import json
import os


def load_env_variables(config_file):
    """
    Load environment variables from a JSON configuration file.

    Args:
        config_file (str): Path to the JSON configuration file.

    Returns:
        dict: A dictionary containing the environment variables.
    """
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"The configuration file {config_file} does not exist.")
    
    try:
        with open(config_file, 'r') as file:
            env_variables = json.load(file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON from the configuration file {config_file}: {e}")
    
    for key, value in env_variables.items():
        # Convert non-string values to strings before setting them in os.environ
        if isinstance(value, dict):
            value = json.dumps(value)  # Convert dictionaries to JSON strings
        os.environ[key] = str(value)
    
    return env_variables

# Example usage
config_file = "/home/user/.aicore/config.json"
try:
    env_variables = load_env_variables(config_file)
    print(f"Loaded environment variables: {env_variables}")
except (FileNotFoundError, ValueError) as e:
    print(e)

In [ ]:
# A0.2 Test connection with env-Variables to SAP AI core

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

In [ ]:
# A0.3 Setup and test connection to HANA DB

import os
# from hana_ml import ConnectionContext
from hdbcli import dbapi

# Fetch environment variables
hdb_host_address = os.getenv("hdb_host_address")
hdb_user = os.getenv("hdb_user")
hdb_password = os.getenv("hdb_password")
hdb_port = os.getenv("hdb_port")

# Debugging: Print non-sensitive environment variables
print(f"hdb_host_address: {hdb_host_address}")
print(f"hdb_user: {hdb_user}")
print(f"hdb_port: {hdb_port}")

# Ensure variables are defined
if not all([hdb_host_address, hdb_user, hdb_password, hdb_port]):
    raise ValueError("One or more HANA DB connection parameters are missing.")

# Convert port to integer
hdb_port = int(hdb_port)

# Create a connection to the HANA database
# hana_connection = ConnectionContext(
#     address=hdb_host_address,
#     port=hdb_port,
#     user=hdb_user,
#     password=hdb_password,
#     encrypt=True
# )

# Test the connection
# print("HANA DB Version:", hana_connection.hana_version())
# print("Current Schema:", hana_connection.get_current_schema())

hana_connection = dbapi.connect(
    address=hdb_host_address,
    port=hdb_port,
    user=hdb_user,
    password=hdb_password,
    #encrypt=True
    autocommit=True,
    sslValidateCertificate=False,
)





In [ ]:
#A0.4 Setup LLM-Connection to SAP AI-HUB

import os
import dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAI

# Lade aicore_model_name aus der Umgebungskonfiguration
aicore_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL"))

# Überprüfe, ob die Variable definiert ist
if not aicore_model_name:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")

llm = ChatOpenAI(proxy_model_name=aicore_model_name)
#llm = OpenAI(proxy_model_name=aicore_model_name)

if not llm:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")
else:
    print(f"""Parameter LLM-Model-Name: {aicore_model_name} wurde erfolgreich geladen.""")


### processing functions Modul A

- function A2: load the pdf-file with accounting assignment guide data and splitt the data into text_chunk

- function A3: vectorize the splitted data with embedding function 

- function A4.1: create a LangChain VectorStore interface for the HANA database and specify the table

- function A4.2: delete existing documents from the table and load embeddings to SAP HANA-Tabele



In [ ]:
# function A2: load the pdf-file containing the accounting assignment guide data from a folder and splitt the data into text_chunks
# the funktion uses the os modules to read the file and split the data with langChain-modules

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def load_and_split_pdf(file_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # Load the PDF file
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    # Split the documents into text_chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(documents)
    
    return text_chunks

if __name__ == "__main__":
    
    # Test the function with a sample PDF file path
    file_path = "data/sample_accounting_guide.pdf"
    try:
        text_chunks = load_and_split_pdf(file_path)
        print(f"Number of text_chunks created: {len(text_chunks)}")
        print(f"First text_chunks: {text_chunks[0]}")
    except FileNotFoundError as e:
        print(e)


In [ ]:
# function A3 - vectorize the splitted data with embedding function 
# Load the text file containing the accounting assignment guide data from a folder and splitt the data into chunks. 
# Use the embedding models from SAP AI-hub for embedding.

# Initialize embeddings

from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

ai_core_embedding_model_name = str(os.getenv('AICORE_DEPLOYMENT_MODEL_EMBEDDING'))
 
try:
    embeddings = init_embedding_model(ai_core_embedding_model_name)
    print("Embedding model initialized successfully.")
except Exception as e:
    print("Embedding model not initialized.")
    print(e)


In [ ]:
# function A4.1 - Create a LangChain VectorStore interface for the HANA database and specify the table (collection) 
# to use for accessing the vector embeddings
# check table creation with sap-hana-database explorer: select * from ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN

from langchain_community.vectorstores.hanavector import HanaDB

vector_table_name = str(os.getenv('hdb_table_name')) + "_" + str(os.getenv('hdb_user')) 

hana_database = HanaDB(
    embedding = embeddings, 
    connection = hana_connection, 
    table_name = vector_table_name
)

try:
    print(f"""
    Successfully created SAP HANA VectorStore interface: {hana_database.connection}
    and SAP HANA table: {vector_table_name}.
    """)
except Exception as e:
    print(e)


In [ ]:
# function A4.2 - delete existing documents from the table and load embeddings to SAP HANA-Table

# Delete already existing documents from the SAP HANA table
hana_database.delete(filter={})

# add the loaded document text_chunks
hana_database.add_documents(text_chunks)

print(f"Successfully added {len(text_chunks)} document chunks to the database.")
print("table-name: ",hana_database.table_name)
print("Successfully connected to the HANA Cloud database.")

In [ ]:
# check function A4.2 - query to the table to verify embeddings

cursor = hana_connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{hana_database.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

print(vectors[:1])

# for vector in vectors:
#     print(vector)

# Check retrievaleval for the embeddings in the SAP-Hana-Database

This code snippet integrates various components from the langchain library to create a retrieval-based question-answering (QA) system. Here's a breakdown of the key parts and their functionality:

Retriever Initialization: The db.as_retriever function is used to initialize a retriever object with specific search arguments ('k':20), which likely defines the number of search results to consider.

Prompt Template : The PromptTemplate was defined in the previous step that instructs how to use the context to answer a question. It emphasizes not to fabricate answers if the information is unavailable. The template also outlines the structure for the expected JSON output with various product and supplier details.The prompt template is crucial for guiding the model's responses, ensuring that the answers are relevant and accurate based on the retrieved information.

Once the retriever and prompt template are set up, the next step involves using the LLM (Language Model) to generate answers based on the retrieved documents. This process typically includes passing the retrieved context to the LLM along with the user's query, allowing it to formulate a coherent and contextually appropriate response.After that, the LLM processes the prompt and generates a response, which can then be formatted and returned to the user, ensuring a seamless interaction with the QA system.

In [ ]:
# function A4.3: Check retrievaleval for the embeddings in the SAP-Hana-Database

import os
from langchain import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

print(llm_chain.invoke({'question': question}))

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce"

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

map_template = """
Analysiere den folgenden Kontext und extrahiere relevante Informationen zur Kontierung:

{context}

Frage: {question}

Gib die relevanten Informationen in einem kurzen Zwischenergebnis zurück.

"""

reduce_template = """
Basierend auf den folgenden Zwischenergebnissen, erstelle eine finale Antwort:

{summerization}

Frage: {question}

Formatiere die Ergebnisse in einer Liste von JSON-Elementen mit den folgenden Schlüsseln:
"Geschäftsfall"
"Konto Soll"
"Konto Haben"

Die Ergebnisse dürfen keine json markdown codeblock syntax enthalten.
Wenn keine relevanten Informationen gefunden wurden, gib an dass Du keine Antwort kennst.
"""

MAP_PROMPT = PromptTemplate(template=map_template, input_variables=["context", "question"])
REDUCE_PROMPT = PromptTemplate(template=reduce_template, input_variables=["summerization", "question"])

chain_type_kwargs = {
    "question_prompt": MAP_PROMPT,
    "reduce_prompt": REDUCE_PROMPT
}

question = "Finde Kontierung für die Umbuchung von langfristigen Forderungen"

retriever = hana_database.as_retriever(search_kwargs={'k':20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='map_reduce',
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="stuff"
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """Verwende den folgenden Kontext, um die Frage am Ende zu beantworten. Wenn du die Antwort nicht kennst,
    sage einfach, dass du es nicht weißt. Versuche nicht, eine Antwort zu erfinden. Formatiere die Ergebnisse als Liste von JSON-Elementen mit den folgenden Schlüsseln:

    "Geschäftsfall",
    "Konto Soll",
    "Konto Haben"

    Füge keine JSON-Markdown-Codeblock-Syntax in die Ergebnisse ein.

    {context}

    Frage: {question}
"""

PROMPT = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )
    
chain_type_kwargs = {"prompt": PROMPT}

question = "Finde Kontierung für die Buchung von Rückstellungen"

count_retrieved_documents = 10

retriever = hana_database.as_retriever(search_kwargs={'k': count_retrieved_documents})
# hint: k smaller than 20 -> to much tokens

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce" (prompt-finetuning)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt für die Map-Phase
map_prompt_template = """Analysiere den folgenden Kontext und extrahiere relevante Kontierungsinformationen für den Geschäftsfall.
Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".

{context}

Frage: {question}
"""

# Prompt für die Combine/Reduce-Phase
combine_prompt_template = """
- Fasse die folgenden Kontierungsinformationen zusammen und entferne Duplikate.
- Gib nur die relevantesten und eindeutigsten Kontierungen zurück. 
- Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".
- Wenn relevante Informationen gefunden wurden gib diese Informationen im folgenden Struktur aus:
    ## Geschäftsfall: <Bezeichnung des Geschäftsfalls>
    ## Kontierung
    Konto-Soll: <Konto-Soll> - <Bezeichnung Konto-Soll> AN Konto-Haben: <Konto-Haben> - <Bezeichnung Konto-Haben>

Beispiel:

-----------------------------
## Geschäftsfall: Bildung von Rückstellungen
    ## Kontierung
    Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen
-----------------------------

{summaries}

Frage: {question}
"""

# Korrekte Struktur für chain_type_kwargs
chain_type_kwargs = {
    "question_prompt": PromptTemplate(
        template=map_prompt_template,
        input_variables=["context", "question"]
    ),
    "combine_prompt": PromptTemplate(
        template=combine_prompt_template,
        input_variables=["summaries", "question"]
    )
}

question = "Finde Kontierung für die Buchung von Bildung von Rückstellungen"

retriever = hana_database.as_retriever(search_kwargs={'k': 20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

example setup SAP HANA Vector Database

Wichtige Aspekte des Codes:
Tabellenstruktur:
ID: Eindeutiger Identifier für jeden Eintrag
DOCUMENT: Der eigentliche Dokumententext
METADATA: JSON-formatierte Metadaten
EMBEDDING: Der Embedding-Vektor als BLOB
VECTOR_DIMENSION: Dimension des Embedding-Vektors (1536 für ada-002)
Vector-Index:
Verwendet HANA's native Vektorindexierung
Cosine-Similarity mit Threshold 0.75
Optimiert für 1536-dimensionale Vektoren
Testdaten:
Beispieldaten für Kontierungsregeln
Dummy-Embeddings für Testzwecke
Strukturierte Metadaten im JSON-Format
Verifikation:
Überprüft Tabellenstruktur
Zeigt vorhandene Indizes
Gibt Anzahl der Datensätze aus
Um den Code zu verwenden:
1. Stellen Sie sicher, dass die Umgebungsvariablen gesetzt sind
2. Führen Sie das Setup aus

In [ ]:
import os
os.environ['HDB_HOST'] = 'your_host'
os.environ['HDB_PORT'] = 'your_port'
os.environ['HDB_USER'] = 'your_user'
os.environ['HDB_PASSWORD'] = 'your_password'
os.environ['HDB_SCHEMA'] = 'your_schema'

In [ ]:
# Tabelle erstellen und Testdaten einfügen
create_vector_table(connection_params)
insert_test_data(connection_params)
verify_table_setup(connection_params)

In [ ]:
# exmaple setup code SAP HANA Vector DB (cursor)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from hdbcli import dbapi
import numpy as np

def create_vector_table(connection_params, table_name="VECTOR_TABLE"):
    """
    Erstellt eine Vektortabelle in SAP HANA für die Speicherung von Dokumenten und deren Embeddings.
    """
    try:
        # Verbindung zur HANA-Datenbank herstellen
        conn = dbapi.connect(
            address=connection_params['address'],
            port=connection_params['port'],
            user=connection_params['user'],
            password=connection_params['password']
        )
        
        cursor = conn.cursor()
        
        # Zum angegebenen Schema wechseln
        if 'schema' in connection_params:
            cursor.execute(f"SET SCHEMA {connection_params['schema']}")

        # Tabelle erstellen, falls sie nicht existiert
        create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            ID NVARCHAR(100) PRIMARY KEY,
            DOCUMENT NCLOB,
            METADATA NCLOB,
            EMBEDDING BLOB,
            VECTOR_DIMENSION INTEGER
        )
        """
        cursor.execute(create_table_sql)
        
        # Vector-Index erstellen
        create_index_sql = f"""
        CREATE INDEX IF NOT EXISTS IDX_{table_name}_VECTOR 
        ON {table_name}(EMBEDDING) 
        VECTOR DIMENSION 1536 
        DOUBLE COSINE THRESHOLD 0.75
        """
        cursor.execute(create_index_sql)
        
        print(f"Tabelle {table_name} und Vector-Index wurden erfolgreich erstellt.")
        
        # Überprüfen, ob die Tabelle leer ist
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"Anzahl der Einträge in der Tabelle: {count}")
        
    except Exception as e:
        print(f"Fehler beim Erstellen der Tabelle: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

def insert_test_data(connection_params, table_name="VECTOR_TABLE"):
    """
    Fügt Testdaten in die Vektortabelle ein.
    """
    try:
        conn = dbapi.connect(
            address=connection_params['address'],
            port=connection_params['port'],
            user=connection_params['user'],
            password=connection_params['password']
        )
        
        cursor = conn.cursor()
        
        if 'schema' in connection_params:
            cursor.execute(f"SET SCHEMA {connection_params['schema']}")

        # Beispiel-Kontierungsdaten
        test_data = [
            {
                'id': 'doc1',
                'document': 'Buchung von Rückstellungen für Garantieverpflichtungen',
                'metadata': '{"type": "accounting_rule", "category": "provisions"}',
                'embedding': np.random.rand(1536).astype(np.float64)  # Dummy-Embedding
            },
            {
                'id': 'doc2',
                'document': 'Anlagenzugang durch Kauf einer Maschine',
                'metadata': '{"type": "accounting_rule", "category": "fixed_assets"}',
                'embedding': np.random.rand(1536).astype(np.float64)
            }
        ]

        # Daten einfügen
        for data in test_data:
            insert_sql = f"""
            INSERT INTO {table_name} 
            (ID, DOCUMENT, METADATA, EMBEDDING, VECTOR_DIMENSION) 
            VALUES(?, ?, ?, ?, 1536)
            """
            
            cursor.execute(insert_sql, 
                         (data['id'], 
                          data['document'], 
                          data['metadata'], 
                          data['embedding'].tobytes()))

        conn.commit()
        print(f"Testdaten wurden erfolgreich in {table_name} eingefügt.")
        
    except Exception as e:
        print(f"Fehler beim Einfügen der Testdaten: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

def verify_table_setup(connection_params, table_name="VECTOR_TABLE"):
    """
    Überprüft die Einrichtung der Vektortabelle.
    """
    try:
        conn = dbapi.connect(
            address=connection_params['address'],
            port=connection_params['port'],
            user=connection_params['user'],
            password=connection_params['password']
        )
        
        cursor = conn.cursor()
        
        if 'schema' in connection_params:
            cursor.execute(f"SET SCHEMA {connection_params['schema']}")

        # Tabellenstruktur überprüfen
        cursor.execute(f"""
        SELECT COLUMN_NAME, DATA_TYPE_NAME, LENGTH, IS_NULLABLE 
        FROM TABLE_COLUMNS 
        WHERE TABLE_NAME = '{table_name.upper()}'
        """)
        
        print("\nTabellenstruktur:")
        for col in cursor.fetchall():
            print(f"Spalte: {col[0]}, Typ: {col[1]}, Länge: {col[2]}, Nullable: {col[3]}")

        # Index überprüfen
        cursor.execute(f"""
        SELECT INDEX_NAME, CONSTRAINT 
        FROM INDEXES 
        WHERE TABLE_NAME = '{table_name.upper()}'
        """)
        
        print("\nIndizes:")
        for idx in cursor.fetchall():
            print(f"Index: {idx[0]}, Constraint: {idx[1]}")

        # Datenbestand überprüfen
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"\nAnzahl der Datensätze: {count}")

    except Exception as e:
        print(f"Fehler bei der Überprüfung: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Verwendung:
if __name__ == "__main__":
    # Verbindungsparameter (sollten aus Umgebungsvariablen kommen)
    connection_params = {
        'address': os.getenv('HDB_HOST'),
        'port': os.getenv('HDB_PORT'),
        'user': os.getenv('HDB_USER'),
        'password': os.getenv('HDB_PASSWORD'),
        'schema': os.getenv('HDB_SCHEMA')
    }

    table_name = "VECTOR_TABLE"

    try:
        # Tabelle erstellen
        create_vector_table(connection_params, table_name)
        
        # Optional: Testdaten einfügen
        insert_test_data(connection_params, table_name)
        
        # Setup überprüfen
        verify_table_setup(connection_params, table_name)
        
    except Exception as e:
        print(f"Fehler beim Setup: {str(e)}")

example insert documentws in SAP HANA

Geschäftsfall          | Konto_Soll | Konto_Haben | Beschreibung
---------------------- | ----------- | ----------- | ------------
Rückstellung_Garantie  | 6815       | 3050        | Buchung von Rückstellungen für Garantieverpflichtungen...
Anlagenzugang_Kauf    | 0410       | 2800        | Anschaffung einer neuen Produktionsmaschine...

Die wichtigsten Features:
Excel-Verarbeitung:
Liest Kontierungsregeln aus Excel
Unterstützt strukturierte Daten mit Geschäftsfall, Konten und Beschreibung
Embedding-Erstellung:
Verwendet OpenAI's ada-002 Modell
Verarbeitet Dokumente in Batches
Fortschrittsanzeige mit tqdm
Datenbankintegration:
Sichere Verbindungshandhabung
Effiziente Batch-Verarbeitung
Fehlerbehandlung und Logging
Metadaten-Handling:
Strukturierte Speicherung der Kontierungsinformationen
JSON-Format für flexible Erweiterbarkeit
Verwendung:
Excel-Datei vorbereiten:

In [ ]:
# Beispiel für manuelle Datenerstellung
import pandas as pd

data = {
    'Geschäftsfall': ['Rückstellung_Garantie', 'Anlagenzugang_Kauf'],
    'Konto_Soll': ['6815', '0410'],
    'Konto_Haben': ['3050', '2800'],
    'Beschreibung': [
        'Buchung von Rückstellungen für Garantieverpflichtungen...',
        'Anschaffung einer neuen Produktionsmaschine...'
    ]
}

df = pd.DataFrame(data)
df.to_excel('kontierungsregeln.xlsx', index=False)

In [ ]:
# Processor initialisieren und ausführen
processor = DocumentProcessor(connection_params)
processor.process_and_store_documents('kontierungsregeln.xlsx')

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import List, Dict
import json
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from tqdm import tqdm
import uuid

class DocumentProcessor:
    def __init__(self, connection_params: Dict, table_name: str = "VECTOR_TABLE"):
        """
        Initialisiert den Document Processor.
        
        Args:
            connection_params: Dictionary mit HANA-Verbindungsparametern
            table_name: Name der Vektortabelle
        """
        self.connection_params = connection_params
        self.table_name = table_name
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-ada-002"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        
    def process_excel_data(self, excel_file: str, sheet_name: str = "Sheet1") -> List[Dict]:
        """
        Verarbeitet Excel-Daten mit Kontierungsregeln.
        
        Args:
            excel_file: Pfad zur Excel-Datei
            sheet_name: Name des Excel-Sheets
        
        Returns:
            Liste von Dokumenten mit Metadaten
        """
        try:
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            documents = []
            
            for _, row in df.iterrows():
                # Annahme: Excel-Spalten sind "Geschäftsfall", "Konto_Soll", "Konto_Haben", "Beschreibung"
                doc = {
                    'id': str(uuid.uuid4()),
                    'document': row['Beschreibung'],
                    'metadata': json.dumps({
                        'Geschäftsfall': row['Geschäftsfall'],
                        'Konto_Soll': str(row['Konto_Soll']),
                        'Konto_Haben': str(row['Konto_Haben'])
                    }, ensure_ascii=False)
                }
                documents.append(doc)
            
            return documents
            
        except Exception as e:
            print(f"Fehler beim Verarbeiten der Excel-Datei: {str(e)}")
            raise

    def create_embeddings(self, documents: List[Dict]) -> List[Dict]:
        """
        Erstellt Embeddings für die Dokumente.
        
        Args:
            documents: Liste von Dokumenten
        
        Returns:
            Liste von Dokumenten mit Embeddings
        """
        try:
            print("Erstelle Embeddings...")
            for doc in tqdm(documents):
                # Embedding für den Dokumententext erstellen
                embedding = self.embeddings.embed_query(doc['document'])
                doc['embedding'] = embedding
            return documents
            
        except Exception as e:
            print(f"Fehler beim Erstellen der Embeddings: {str(e)}")
            raise

    def insert_documents(self, documents: List[Dict]):
        """
        Fügt Dokumente in die HANA-Datenbank ein.
        
        Args:
            documents: Liste von Dokumenten mit Embeddings
        """
        from hdbcli import dbapi
        import numpy as np
        
        try:
            conn = dbapi.connect(
                address=self.connection_params['address'],
                port=self.connection_params['port'],
                user=self.connection_params['user'],
                password=self.connection_params['password']
            )
            
            cursor = conn.cursor()
            
            if 'schema' in self.connection_params:
                cursor.execute(f"SET SCHEMA {self.connection_params['schema']}")

            print("Füge Dokumente in die Datenbank ein...")
            for doc in tqdm(documents):
                insert_sql = f"""
                INSERT INTO {self.table_name} 
                (ID, DOCUMENT, METADATA, EMBEDDING, VECTOR_DIMENSION) 
                VALUES(?, ?, ?, ?, 1536)
                """
                
                # Embedding in bytes konvertieren
                embedding_bytes = np.array(doc['embedding']).astype(np.float64).tobytes()
                
                cursor.execute(insert_sql, 
                             (doc['id'], 
                              doc['document'], 
                              doc['metadata'], 
                              embedding_bytes))

            conn.commit()
            print(f"Alle Dokumente wurden erfolgreich eingefügt.")
            
        except Exception as e:
            print(f"Fehler beim Einfügen der Dokumente: {str(e)}")
            raise
        finally:
            if 'cursor' in locals():
                cursor.close()
            if 'conn' in locals():
                conn.close()

    def process_and_store_documents(self, excel_file: str, sheet_name: str = "Sheet1"):
        """
        Hauptmethode zum Verarbeiten und Speichern von Dokumenten.
        
        Args:
            excel_file: Pfad zur Excel-Datei
            sheet_name: Name des Excel-Sheets
        """
        try:
            # Dokumente aus Excel laden
            documents = self.process_excel_data(excel_file, sheet_name)
            print(f"Anzahl geladener Dokumente: {len(documents)}")
            
            # Embeddings erstellen
            documents_with_embeddings = self.create_embeddings(documents)
            
            # Dokumente in die Datenbank einfügen
            self.insert_documents(documents_with_embeddings)
            
        except Exception as e:
            print(f"Fehler bei der Verarbeitung: {str(e)}")
            raise

# Beispiel für die Verwendung:
if __name__ == "__main__":
    # Verbindungsparameter
    connection_params = {
        'address': os.getenv('HDB_HOST'),
        'port': os.getenv('HDB_PORT'),
        'user': os.getenv('HDB_USER'),
        'password': os.getenv('HDB_PASSWORD'),
        'schema': os.getenv('HDB_SCHEMA')
    }

    # Excel-Datei mit Kontierungsregeln
    excel_file = "kontierungsregeln.xlsx"

    # Document Processor initialisieren und ausführen
    processor = DocumentProcessor(connection_params)
    
    try:
        processor.process_and_store_documents(excel_file)
    except Exception as e:
        print(f"Fehler beim Gesamtprozess: {str(e)}")